In [1]:
import requests
import pandas as pd
import io
 
states_to_fetch = [ 
    {"name": "Andhra Pradesh", "lat": 16.5062, "lon": 80.6480},
    {"name": "Arunachal Pradesh", "lat": 27.1004, "lon": 93.6056},
    {"name": "Assam", "lat": 26.1445, "lon": 91.7362},
    {"name": "Bihar", "lat": 25.5941, "lon": 85.1376},
    {"name": "Chhattisgarh", "lat": 21.2514, "lon": 81.6296},
    {"name": "Goa", "lat": 15.4909, "lon": 73.8278},
    {"name": "Gujarat", "lat": 23.2156, "lon": 72.6369},
    {"name": "Haryana", "lat": 30.7333, "lon": 76.7794},
    {"name": "Himachal Pradesh", "lat": 31.1048, "lon": 77.1734},
    {"name": "Jharkhand", "lat": 23.3441, "lon": 85.3096},
    {"name": "Karnataka", "lat": 12.9716, "lon": 77.5946},
    {"name": "Kerala", "lat": 8.5241, "lon": 76.9366},
    {"name": "Madhya Pradesh", "lat": 23.2599, "lon": 77.4126},
    {"name": "Maharashtra", "lat": 19.0760, "lon": 72.8777},
    {"name": "Manipur", "lat": 24.8170, "lon": 93.9368},
    {"name": "Meghalaya", "lat": 25.5788, "lon": 91.8933},
    {"name": "Mizoram", "lat": 23.7367, "lon": 92.7176},
    {"name": "Nagaland", "lat": 25.6751, "lon": 94.1086},
    {"name": "Odisha", "lat": 20.2961, "lon": 85.8245},
    {"name": "Punjab", "lat": 30.7333, "lon": 76.7794},
    {"name": "Rajasthan", "lat": 26.9124, "lon": 75.7873},
    {"name": "Sikkim", "lat": 27.3389, "lon": 88.6065},
    {"name": "Tamil Nadu", "lat": 13.0827, "lon": 80.2707},
    {"name": "Telangana", "lat": 17.3850, "lon": 78.4867},
    {"name": "Tripura", "lat": 23.8315, "lon": 91.2868},
    {"name": "Uttar Pradesh", "lat": 26.8467, "lon": 80.9462},
    {"name": "Uttarakhand", "lat": 30.3165, "lon": 78.0322},
    {"name": "West Bengal", "lat": 22.5726, "lon": 88.3639},
 
    {"name": "Andaman and Nicobar Islands", "lat": 11.6234, "lon": 92.7265},
    {"name": "Chandigarh", "lat": 30.7333, "lon": 76.7794},
    {"name": "Dadra and Nagar Haveli and Daman and Diu", "lat": 20.4283, "lon": 72.8397},
    {"name": "Delhi", "lat": 28.6139, "lon": 77.2090},
    {"name": "Jammu and Kashmir", "lat": 34.0837, "lon": 74.7973},
    {"name": "Ladakh", "lat": 34.1526, "lon": 77.5771},
    {"name": "Lakshadweep", "lat": 10.5667, "lon": 72.6417},
    {"name": "Puducherry", "lat": 11.9416, "lon": 79.8083}
] 
api_parameters = [
"CLOUD_AMT",
"AOD_55",
"PW",
"QV2M",
"T2M",
"RH2M",
"SZA",
"WS50M",
"PS",
"TOA_SW_DWN",
"ALLSKY_SFC_SW_DWN"
    
] 
base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
start_date = "20150101"   
end_date = "20241231"    
 
for state in states_to_fetch:
    print(f"Fetching data for {state['name']}...")
    
    params = {
        "parameters": ",".join(api_parameters),
        "community": "RE",
        "longitude": state['lon'],
        "latitude": state['lat'],
        "start": start_date,
        "end": end_date,
        "format": "CSV"   
    }
    
    try:
        response = requests.get(base_url, params=params, timeout=150) 
         
        if response.status_code == 200: 
            content = response.text
            data_start_index = content.find("YEAR,MO,DY")
            
            if data_start_index != -1: 
                csv_data = content[data_start_index:]
                df = pd.read_csv(io.StringIO(csv_data))
                 
                if df.empty:
                    print(f"No data returned for {state['name']}. Skipping.")
                    continue 
                df['Date'] = pd.to_datetime(df[['YEAR', 'MO', 'DY']].astype(str).agg('-'.join, axis=1))
                 
                df['State'] = state['name']
                df['Latitude'] = state['lat']
                df['Longitude'] = state['lon']
                 
                df = df.drop(columns=['YEAR', 'MO', 'DY'])
                cols_to_move = ['Date', 'State', 'Latitude', 'Longitude']
                df = df[cols_to_move + [col for col in df.columns if col not in cols_to_move]]
                 
                filename = f"{state['name']}_raw.csv"
                df.to_csv(filename, index=False)
                print(f"Successfully saved {filename}")
                
            else:
                print(f"Could not parse CSV data for {state['name']}. Response: {content}")
        
        else:
            print(f"Error fetching data for {state['name']}. Status Code: {response.status_code}")
            print(f"Response: {response.text}")

    except requests.exceptions.Timeout:
        print(f"Request for {state['name']} timed out. Skipping.")
    except Exception as e:
        print(f"An error occurred for {state['name']}: {e}")

print("Data fetching complete.")

Fetching data for Andhra Pradesh...
Successfully saved Andhra Pradesh_raw.csv
Fetching data for Arunachal Pradesh...
Successfully saved Arunachal Pradesh_raw.csv
Fetching data for Assam...
Successfully saved Assam_raw.csv
Fetching data for Bihar...
Successfully saved Bihar_raw.csv
Fetching data for Chhattisgarh...
Successfully saved Chhattisgarh_raw.csv
Fetching data for Goa...
Successfully saved Goa_raw.csv
Fetching data for Gujarat...
Successfully saved Gujarat_raw.csv
Fetching data for Haryana...
Successfully saved Haryana_raw.csv
Fetching data for Himachal Pradesh...
Successfully saved Himachal Pradesh_raw.csv
Fetching data for Jharkhand...
Successfully saved Jharkhand_raw.csv
Fetching data for Karnataka...
Successfully saved Karnataka_raw.csv
Fetching data for Kerala...
Successfully saved Kerala_raw.csv
Fetching data for Madhya Pradesh...
Successfully saved Madhya Pradesh_raw.csv
Fetching data for Maharashtra...
Successfully saved Maharashtra_raw.csv
Fetching data for Manipur...
S

In [2]:
import pandas as pd
import glob
 
all_files = glob.glob("*.csv")
merged_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
merged_df.to_csv("Solar.csv", index=False)
print("Merged", len(all_files), "files into Solar.csv")


Merged 38 files into Solar.csv
